# Reduce fMRIPrep derivatives to 60 volumes

In [1]:
import os
from glob import glob

import nibabel as nb
import pandas as pd

In [2]:
fmriprep_dir = "data/fmriprep-reduced/"
func_dir = os.path.abspath(os.path.join(fmriprep_dir, "sub-1648798153/ses-PNC1/func/"))

In [3]:
nii_files = sorted(glob(os.path.join(func_dir, "*_bold.nii.gz")))
for nii_file in nii_files:
    nii_img = nb.load(nii_file)
    nii_data = nii_img.get_fdata()
    nii_data = nii_data[..., :60]
    nii_img = nb.Nifti1Image(nii_data, nii_img.affine, nii_img.header)
    nii_img.to_filename(nii_file)

In [4]:
cii_files = sorted(glob(os.path.join(func_dir, "*_bold.dtseries.nii")))
for cii_file in cii_files:
    cii_img = nb.load(cii_file)
    cii_data = cii_img.get_fdata()
    cii_data = cii_data[:60, :]
    time_axis, brain_model_axis = [cii_img.header.get_axis(i) for i in range(cii_img.ndim)]
    time_axis = time_axis[:60]
    header = nb.cifti2.Cifti2Header.from_axes((time_axis, brain_model_axis))
    cii_img = nb.Cifti2Image(cii_data, header=header, nifti_header=cii_img.nifti_header)
    cii_img.to_filename(cii_file)

In [5]:
tsv_files = sorted(glob(os.path.join(func_dir, "*_timeseries.tsv")))
for tsv_file in tsv_files:
    tsv_df = pd.read_table(tsv_file)
    tsv_df = tsv_df.loc[:59]
    tsv_df.to_csv(tsv_file, sep="\t", index=False, na_rep="n/a")